In [ ]:
!pip install pymysql

In [ ]:
conda install -c anaconda pymysql  # mac 에서 잘 안돼서 다시 설치

In [ ]:
import pymysql
conn = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = '',
    db = 'test',
    charset = 'utf8mb4'
)

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT * FROM sales") # 숫자 2가 나오면 잘 된것을 알 수 있음(다른 숫자 나올수도)

In [ ]:
cur.execute("""CREATE TABLE customer(
                ID INT PRIMARY KEY AUTO_INCREMENT,
                name VARCHAR(50) NOT NULL,
                email VARCHAR(200))""")

In [ ]:
cur.execute("INSERT INTO customer (name) VALUES ('임선미')")

In [ ]:
conn.commit() # commit 을 해줘야 확실히 값이 들어감

In [ ]:
while (True):
    name = input("사용자 이름은?")
    if name == "":
        print("사용자 이름은 필수 입력값입니다.")
        continue
    email = input("사용자의 이메일 주소를 입력해주세요")
    text = "INSERT INTO customer (name,email) VALUES('"+ name + "','" + email +"')"
    cur.execute(text)
    break; # 하나만 입력받게

In [ ]:
cur.execute("SELECT * FROM customer")

print("ID       이름         이메일")
print("-------------------------")

while (True) :
    row = cur.fetchone() # 데이터를 한행씩
    if row == None:
        break
    cus_id = row[0]
    name = row[1]
    email = row[2]
    print("%2s %10s %10s"%(cus_id, name, email))

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
cur.execute("SELECT * FROM customer")
result = cur.fetchall()                        # 데이터 가져오기
columns = [col[0] for col in cur.description]  # column 명을 가져오기 위해 / sql의 describe customer; 과 같음 
df = pd.DataFrame(result, columns = columns)   # 파이썬 데이터 프레임 이용해 가져옴
df

In [ ]:
# world db 에 연결
conn = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = '',
    db = 'world',
    charset = 'utf8mb4'
)

In [ ]:
cur = conn.cursor()

In [ ]:
# country 테이블에서
# GovernmentForm 필드 기준으로 내림차순으로 정렬한 후
# SurfaceArea 필드 기준으로 오름차순 정렬하여 조회
import pandas as pd
cur.execute("SELECT * FROM country")      # world db의 country 테이블을 읽어옴
result = cur.fetchall()
columns = [col[0] for col in cur.description]
df = pd.DataFrame(result, columns = columns)
df.head()

In [ ]:
# GovernmentForm 필드 기준으로 내림차순으로 정렬한 후
# SurfaceArea 필드 기준으로 오름차순 정렬하여 조회
# python 이용해봄
df_sorted = df.sort_values(by =['GovernmentForm', 'SurfaceArea'], ascending = [False, True])
df_sorted.head()

In [ ]:
# country 테이블에서
# Continent 필드가 'Africa'인 데이터 중에
# Population 기준으로 오름차순 정렬한 후
# Name, Population 데이터

cur.execute(""" SELECT 
                    Name, Population
                FROM country
                WHERE Continent = 'Africa'
                ORDER BY Population
                """)
result = cur.fetchall()       # 데이터 가져옴
columns = [col[0] for col in cur.description]  # 열 속성들 가져옴
df = pd.DataFrame(result,columns=columns)
df.head()

In [ ]:
# country 테이블에서 Continent 필드로 그룹화하고
# GNP 평균값을 소수점 2자로 반올림하여 조회하고 이름을 avg_GNP로 출력

cur.execute(""" SELECT
                    Continent,
                    ROUND(AVG(GNP),2) as avg_GNP
                FROM country
                GROUP BY Continent""")
result =  cur.fetchall()
columns = [col[0] for col in cur.description]
df = pd.DataFrame(result, columns = columns)
df

In [ ]:
# country 테이블에서 Region 필드로 그룹화하고
# 나라의 개수가 15이상 20이하인 데이터를 count_reg로 하여 내림차순으로 조회

cur.execute("""SELECT
                    REGION,
                    COUNT(Region) as count_reg
                FROM country
                GROUP BY Region
                HAVING count_reg >= 15 and count_reg <= 20""")
result =  cur.fetchall()
columns = [col[0] for col in cur.description]
df =  pd.DataFrame(result, columns = columns)
df

In [ ]:
# country 테이블의 각 국가마다
# 해당 국가에 소속된 도시들의 평균 인구를 city 테이블을 이용하여 조회

cur.execute("""SELECT
                  CountryCode,
    			  AVG(Population)
              FROM city
              GROUP BY CountryCode""")
result =  cur.fetchall()
columns = [col[0] for col in cur.description]
df =  pd.DataFrame(result, columns = columns)
df

In [ ]:
# country 테이블의 Frace의 국가코드를 이용하여
# city 테이블의 'France'의 모든 도시의 이름 출력
cur.execute("""SELECT 
    				c.Name
               FROM city c
               JOIN country t
                   ON c.CountryCode = t.Code
               WHERE t.Name = 'France'""")
result =  cur.fetchall()
columns = [col[0] for col in cur.description]
df =  pd.DataFrame(result, columns = columns)
df.head()

In [ ]:
# world DB의 city와 country 테이블과 이용하여
# 도시 인구 비율 계산 함수 (도시인구/해당 국가의 인구)

def city_pop_ratio(city_name):
    cur.execute(f"""SELECT c.Population, t.Population
                   FROM city c JOIN country t ON c.CountryCode = t.Code
                   WHERE c.Name='{city_name}'""")
    city_pop, country_pop = cur.fetchone()
    return round(city_pop/country_pop*100,2)

city_pop_ratio("Seoul")